Connected to base (Python 3.11.5)

In [1]:
import matplotlib.pyplot as plt
from transformers import AutoModelForImageClassification, AutoConfig, TrainingArguments, Trainer, AutoModel, AutoImageProcessor
from utils import read_yaml, getYAMLParameter, HugginfaceProcessorData
from datasets import load_dataset
from torch.utils.data import DataLoader
from logger import OnePixelLogger, CWLogger, PGDLogger
from utils import plot_grid, getYAMLParameter, one_hot_encode
import tqdm
import wandb
import math
import argparse 
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from clip_classifier import clip_classifier


/home/cesar/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/cesar/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/cesar/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
device = torch.device("cuda")
labels_names = ["A","B","C","D","E","F","G","H"]

model  = clip_classifier(clip_model_name="openai/clip-vit-base-patch32",
                                  labels_name=labels_names).to(device)
image_processor = AutoImageProcessor.from_pretrained("openai/clip-vit-base-patch32")


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [4]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor( text=["s","da"], return_tensors="pt")

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities


ValueError: You have to specify pixel_values

In [2]:
len(inputs.data["pixel_values"])

2

In [3]:
inputs.data

{'input_ids': tensor([[49406,   338, 49407],
         [49406,  1140, 49407]]),
 'attention_mask': tensor([[1, 1, 1],
         [1, 1, 1]]),
 'pixel_values': tensor([[[[ 0.5873,  0.5873,  0.6165,  ...,  0.0617,  0.0471, -0.0259],
           [ 0.5727,  0.5727,  0.6603,  ...,  0.1201,  0.0763,  0.0909],
           [ 0.5873,  0.5435,  0.6165,  ...,  0.0325,  0.1201,  0.0617],
           ...,
           [ 1.8719,  1.8573,  1.8719,  ...,  1.3902,  1.4340,  1.4194],
           [ 1.8281,  1.8719,  1.8427,  ...,  1.4486,  1.4340,  1.5070],
           [ 1.8573,  1.9011,  1.8281,  ...,  1.3756,  1.3610,  1.4486]],
 
          [[-1.3169, -1.3019, -1.3169,  ..., -1.4970, -1.4369, -1.4820],
           [-1.2418, -1.2718, -1.2268,  ..., -1.4369, -1.4669, -1.4519],
           [-1.2568, -1.3169, -1.2268,  ..., -1.4669, -1.4069, -1.4519],
           ...,
           [ 0.1239,  0.1089,  0.1239,  ..., -0.7016, -0.6865, -0.6865],
           [ 0.0789,  0.0939,  0.0488,  ..., -0.6565, -0.6865, -0.6115],
       